## MongoDB - Kafka Data Streaming

The following notebook illustrates how to setup continuos streaming of the data from MongoDB via to Databricks Delta Lake

Update the variable **TOPIC** , **KAFKA_BROKER**, **API_KEY** & **SECRET** with relevant details.

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

TOPIC = "<DB.COLLECTION>";
KAFKA_BROKER = "hostname:port1, hostname:port2";  // comma separated list of broker:host
API_KEY = "";
SECRET = "";


raw_kafka_events = (spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_BROKER)  
    .option("subscribe", TOPIC)
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(API_KEY, SECRET))
    .option("kafka.ssl.endpoint.identification.algorithm", "https")
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("failOnDataLoss", "false")
    .option("startingOffsets", "earliest")
    .load())


@dlt.table(table_properties={"pipelines.reset.allowed":"false"})
def mdb_data():
  return raw_kafka_events